# 1-scraping

In [58]:
# Manejo de datos 
import pandas as pd # manejo de datos y dataframes
import numpy as np # manejo de arrays y operaciones matematicas 


# Librerias para realizar web scraping con selenium
from selenium import webdriver # webdriver permite manejar un navegador 
from webdriver_manager.chrome import ChromeDriverManager # permite instalar y mantener actualizado el driver de chrome
from selenium.webdriver.common.keys import Keys # permite simular teclas del teclado 
from selenium.webdriver.chrome.options import Options # permite configurar el driver de chrome como modo incognito o maximizar la ventana
from time import sleep # esperas entre ejecuciones de codigo


import warnings # permite ignorar los warnings de python 
warnings.filterwarnings('ignore')

In [59]:
import requests
import re
import pickle

from bs4 import BeautifulSoup

In [60]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

# para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') # empezar maximizado
opciones.add_argument('user.data-dir=selenium') # guarda las cookies
opciones.add_argument('--incognito') # incognito window

# selenium

In [61]:
urls = []

driver = webdriver.Chrome()

driver.get("https://www.dia.es/") 
sleep(3)

# aceptamos las cookies 
driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
sleep(2)

# clicka productos
driver.find_element("css selector", '#app > div > div > div > div.home-view__header > div.dia-header > div.dia-header__section.dia-header__section--start > div > div > button').click()
sleep(2)

# clicka primera categoría (charcutería y quesos)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[1]').click()
sleep(2)

# charcutería y quesos urls
for i in range (1,11):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[1]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka segunda categoría (carnicería)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[2]/a').click()
sleep(2)

# carnicería urls
for i in range (1,7):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[2]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka tercera categoría (pescados, mariscos y ahumados)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[3]/a').click()
sleep(2)

# pescados, mariscos y ahumados urls
for i in range (1,5):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[3]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka cuarta categoría (verduras)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[4]/a').click()
sleep(2)

# verduras urls
for i in range (1,11):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[4]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka quinta categoría (frutas)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[5]/a').click()
sleep(2)

# frutas urls
for i in range (1,12):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[5]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka sexta categoría (leches)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[6]/a').click()
sleep(2)

# leches urls
for i in range (1,7):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[6]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka séptima categoría (yogur)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[7]/a').click()
sleep(2)

# yogur urls
for i in range (1,13):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[7]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka octava categoría (arroz)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[8]/a').click()
sleep(2)

# arroz urls
for i in range (1,8):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[8]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka novena categoría (aceite)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[9]/a').click()
sleep(2)

# aceite urls
for i in range (1,6):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[9]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka decima categoría (conservas)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[10]/a').click()
sleep(2)

# conservas urls
for i in range (1,11):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[10]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka undecima categoría (panes)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[11]/a').click()
sleep(2)

# panes urls
for i in range (1,8):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[11]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (cafes)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[12]/a').click()
sleep(2)

# cafes urls
for i in range (1,5):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[12]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (azucar)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[13]/a').click()
sleep(2)

# azucar urls
for i in range (1,6):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[13]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (galletas)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[14]/a').click()
sleep(2)

# galletas urls
for i in range (1,6):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[14]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (papas)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[15]/a').click()
sleep(2)

# papas urls
for i in range (1,4):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[15]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (pizza)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[16]/a').click()
sleep(2)

# pizza urls
for i in range (1,5):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[16]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (congelados)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[17]/a').click()
sleep(2)

# congelados urls
for i in range (1,11):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[17]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (agua)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[18]/a').click()
sleep(2)

# agua urls
for i in range (1,12):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[18]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (cerves)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[19]/a').click()
sleep(2)

# cerves urls
for i in range (1,15):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[19]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (hogar)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[20]/a').click()
sleep(2)

# hogar urls
for i in range (1,15):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[20]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (perfumeria)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[21]/a').click()
sleep(2)

# perfumeria urls
for i in range (1,18):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[21]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (bb)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[22]/a').click()
sleep(2)

# bb urls
for i in range (1,7):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[22]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

# clicka duodedima categoría (perro)
driver.find_element("xpath", '//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[23]/a').click()
sleep(2)

# perro urls
for i in range (1,4):
    urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[23]/ul/div[{i}]/a').get_attribute("href"))
sleep(5)

driver.quit()

In [62]:
df = pd.DataFrame(urls, columns=['url'])

df.to_csv("../data/urls-dia.csv", index=False, sep= ",")

# sopa bonita

In [63]:
# ['supermarket', 'category', 'name', 'description', 'price', 'reference_price', 'reference_unit', 'insert_date', 'product_id']

resultados_dia = {"supermarket": 'dia-es',
                "category":[],
                "name":[], 
                "description": [], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": [],
                "product_id": []}

In [64]:
url = 'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001'
res = requests.get(url)
sleep(2)
#driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

print(res)

sopa = BeautifulSoup(res.content, 'html.parser')

productos = sopa.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
#productos = sopa.find_all("div", {"data-test-id": "product-card"})
productos = sopa.find_all("p", {"class": "search-product-card__product-name"})

print(f'Cantidad de productos extraídos: {len(productos)}')

print(productos[0])

print(productos[0].text)


<Response [200]>
Cantidad de productos extraídos: 10
<p class="search-product-card__product-name">Pechuga de pavo NUESTRA ALACENA pack 2 unidades SOBRE 400 GR</p>
Pechuga de pavo NUESTRA ALACENA pack 2 unidades SOBRE 400 GR
